In [1]:
# Importaciones e instalaciones necesarias
from selenium import webdriver
import random 
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # pip install webdriver-manager
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

# Para trabajar con mongodb
!pip install pymongo
from pymongo import MongoClient
client = MongoClient('localhost')
db = client['pruebaTusDatos']
col = db['procesos']
detalles = db['detalles']
actuaciones = db['actuaciones']

In [2]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

# Conexión con la página web a "scrapear"
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=opts)
driver.get('https://procesosjudiciales.funcionjudicial.gob.ec/busqueda-filtros')

# Se establece el tipo de proceso a realizar
#type_process = int(input("Desea ingresar código para: \n"+
 #                  "1. Demandante\n"+
  #                 "2. Demandado\n"))
# Código del demandante o demandado
# Demandante1 = 0968599020001
# Demandante2 = 0992339411001

# Demandado1 = 1791251237001
# Demandado2 = 0968599020001
# codigo = input("Ingrese el número de identificación: ")
type_process = 1
codigo = "0968599020001"


In [3]:
# Me doy cuenta que la pagina carga el formulario dinamicamente luego de que la carga incial ha sido completada
# Por eso tengo que esperar que aparezca 
if type_process == 1:
    input_user = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@formcontrolname = 'cedulaActor']")))
    input_user.send_keys(codigo)
elif type_process == 2:
    input_user = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@formcontrolname = 'cedulaDemandado']")))
    input_user.send_keys(codigo)


# Click al boton buscar
boton_buscar = driver.find_element(By.XPATH,"//button[@type = 'submit']")
boton_buscar.click()


# Trabajo con PROCESO

### Función para enviar un proceso a la base de datos

In [9]:
def enviar_proceso(listado_proceso):
    for proceso in listado_proceso:
        id_proceso = proceso.find_element('xpath','.//div[@class="id"]').text
        fecha = proceso.find_element('xpath','.//div[@class="fecha"]').text
        numero_proceso = proceso.find_element('xpath','.//div[@class="numero-proceso"]').text
        accion_infraccion = proceso.find_element('xpath','.//div[@class="accion-infraccion"]').text
        
        # Se envia la información obtenida a la base de datos
        col.insert_one({
            'id':id_proceso,
            'fecha': fecha,
            'numero_proceso': numero_proceso,
            'accion_infraccion': accion_infraccion
        })

### Obtención cantidad de vistas con procesos

In [10]:
# Se controla la excepción en caso de que la página no cargue correctamente y no se pueda obtener el elemento page_label
try:
    # Se obtiene el número limite para realizar los cambios de página
    page_label = driver.find_element(By.XPATH, "//div[@class = 'mat-mdc-paginator-range-label']")
    text = page_label.text

    # Se utiliza la funcion split para obtener un array por palabra del texto obtenido y luego se obtiene el ultimo registro
    parts = text.split()
    limit = int(parts[len(parts) - 1])
except:
    driver.refresh()

### Obtención de todos los procesos

In [11]:
# Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
# Se envia la lista de procesos encontrados por pantalla a la base de datos
enviar_proceso(listado_procesos)

# Ciclo para recorrer el listado de procesos y poder obtener y enviar los detalles de cada uno
#for proceso in listado_procesos:
 #   obtencion_detalle(proceso)
        

# Se obtiene el boton de paso
boton_paso = driver.find_element(By.XPATH,"//button[@aria-label='Página siguiente']")
                                                           
# Se recorren todas las vistas de los registros
for i in range(limit-1):
    
        # click al boton de paso
        boton_paso.click()
        
        # Espero a que se cargue la nueva página
        sleep(random.uniform(8.0,10.0))
        
        # Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
        listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
        
        # Se envia la lista de procesos encontrados por pantalla a la base de datos
        enviar_proceso(listado_procesos)
        
        #for proceso in listado_procesos:
         #       obtencion_detalle(proceso)
            
            
        # Espero a que se cargue la nueva página
        sleep(random.uniform(8.0,10.0))
        # Se obtiene el boton de paso
        boton_paso = driver.find_element(By.XPATH,"//button[@aria-label='Página siguiente']")

### Retornando a la vista inicial

In [12]:
boton_inicio = driver.find_element(By.XPATH,"//button[@aria-label='Primera página']")
boton_inicio.click()

# Trabajo con DETALLES

### Función para enviar los detalles

In [13]:
def enviar_detalles(numero_proceso,listado_detalles):
    for detalle in listado_detalles:
        id_detalle = detalle.find_element('xpath','.//div[@class = "numero-incidente"]').text
        fecha = detalle.find_element('xpath','.//div[@class = "fecha-ingreso"]').text
        actores_ofendidos = detalle.find_element('xpath','.//div[@class = "lista-actores"]').text
        demandados_procesados = detalle.find_element('xpath','.//div[@class = "lista-demandados"]').text
        
        
        detalles.insert_one({
            'id': id_detalle,
            'numero_proceso': numero_proceso,
            'fecha' : fecha,
            'actores_ofendidos' : actores_ofendidos,
            'demandados_procesados' : demandados_procesados
        })

### Función para obtener los detalles de un proceso

In [14]:
# Obtención del detalle
# Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))

#print(listado_procesos[0].text)
def obtencion_detalle(proceso):
    
        # Se espera unos segundos antes de dar click al boton de detalles
        sleep(random.uniform(8.0,10.0))
        listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
        
        numero_proceso = proceso.find_element(By.XPATH,'.//div[@class="numero-proceso"]').text
        boton_proceso = proceso.find_element(By.XPATH,'.//a[@href="/movimientos"]')
        boton_proceso.click()
        
        #listado_detalles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[contains(@class = 'cabecera-tabla')]")))
        listado_detalles = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, \"lista-movimiento-individual\")]")))

        # Se envian los detalles a la base de datos
        enviar_detalles(numero_proceso,listado_detalles)


### Función para hacer el recorrido de una vista

In [15]:
def recorrido_por_vista(num_vista,listado_procesos):
    for i in range(len(listado_procesos)):
        try:
            sleep(random.uniform(3.0,5.0))
            listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
            sleep(random.uniform(3.0,5.0))

            # Se obtiene el boton de paso
            boton_paso = driver.find_element(By.XPATH,"//button[@aria-label='Página siguiente']")

            # ciclo para dar click en el boton de paso las veces necesarias para llegar a la vista que necesito
            for n in range(num_vista-1) : 
                boton_paso.click()

            sleep(random.uniform(3.0,5.0))
            listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
            #boton_proceso = listado_procesos[i].find_element(By.XPATH,'.//a[@href="/movimientos"]')
            #boton_proceso.click()

            # Obtengo los detalles del proceso
            # sleep(random.uniform(3.0,5.0))
            obtencion_detalle(listado_procesos[i])

            sleep(random.uniform(3.0,5.0))
            driver.back()
            
        except StaleElementReferenceException:
            sleep(random.uniform(3.0,5.0))
            listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))



### Realizando el recorrido por todas las vistas 
##### En este recorrido se ingresa a cada uno de los detalles, se "scrapean" y se envian a la base de datos en la tabla "detalles"

In [16]:
num_vista = 1
sleep(random.uniform(3.0,5.0))
listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))

for i in range(limit-1):
   # sleep(random.uniform(3.0,5.0))
    listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
    recorrido_por_vista(num_vista, listado_procesos)

    num_vista += 1

KeyboardInterrupt: 

# Trabajo con Actuaciones Judiciales

### Función para guardar las actuaciones judiciales en la base de datos

In [6]:
def enviar_actuaciones(numero_proceso, listado_actuaciones):
    for actuacion in listado_actuaciones:
        #fecha_ingreso = detalle.find_element('xpath','.//')
        titulo_detalle = actuacion.find_element('xpath','.//span[contains(@class, \"title\")]').text
        
        actuaciones.insert_one({
            'titulo_detalle': titulo_detalle,
            'numero_proceso': numero_proceso
        })

### Función para obtener las actuaciones de un proceso

In [7]:
# Obtención del detalle
# Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))

#print(listado_procesos[0].text)
def obtencion_actuaciones(proceso):
    #try:
        # Se espera unos segundos antes de dar click al boton de detalles
        sleep(random.uniform(8.0,10.0))
        #listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
        
        numero_proceso = proceso.find_element(By.XPATH,'.//div[@class="numero-proceso"]').text
        boton_proceso = proceso.find_element(By.XPATH,'.//a[@href="/movimientos"]')
        boton_proceso.click()

        # Se espera unos segundos antes de dar click al boton de detalles
        sleep(random.uniform(8.0,10.0))
        boton_detalle_proceso = driver.find_element(By.XPATH,'.//a[@href="/actuaciones"]')
        boton_detalle_proceso.click()
        # Se extraen todos los detalles según la fecha de ingreso
        #listado_detalles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[contains(@class = 'cabecera-tabla')]")))
        listado_actuaciones = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//span[contains(@class, \"mat-content\")]")))

        # Se envian los detalles a la base de datos
        enviar_actuaciones(numero_proceso,listado_actuaciones)

        #sleep(random.uniform(3.0,5.0))
        #driver.back()
        #driver.back()
        
    #except StaleElementReferenceException:
        # Reintentar localizar y obtener detalles del proceso
        #sleep(2)  # Puede ajustarse según necesidades
        #obtencion_actuaciones(proceso)

#obtencion_actuaciones(listado_procesos[0])



### Función para hacer el recorrido de una vista

In [8]:
def recorrido_por_vista(num_vista,listado_procesos):
    for i in range(len(listado_procesos)):
        try:
            sleep(random.uniform(3.0,5.0))
            listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
            sleep(random.uniform(3.0,5.0))

            # Se obtiene el boton de paso
            boton_paso = driver.find_element(By.XPATH,"//button[@aria-label='Página siguiente']")

            # ciclo para dar click en el boton de paso las veces necesarias para llegar a la vista que necesito
            for n in range(num_vista-1) : 
                boton_paso.click()

            sleep(random.uniform(3.0,5.0))
            listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
            #boton_proceso = listado_procesos[i].find_element(By.XPATH,'.//a[@href="/movimientos"]')
            #boton_proceso.click()

            # Obtengo los detalles del proceso
            # sleep(random.uniform(3.0,5.0))
            obtencion_actuaciones(listado_procesos[i])

            sleep(random.uniform(3.0,5.0))
            driver.back()
            driver.back()
            
        except StaleElementReferenceException:
            sleep(random.uniform(3.0,5.0))
            listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))


### Realizando el recorrido por todas las vistas
#### En este recorrido se ingresa a cada una de las actuaciones, se "scrapean" y se envian a la base de datos en la tabla "actuaciones"

In [11]:
num_vista = 1
sleep(random.uniform(3.0,5.0))
listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))

for i in range(limit-1):
    listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
    recorrido_por_vista(num_vista, listado_procesos)
    
    num_vista += 1

ElementClickInterceptedException: Message: element click intercepted: Element <button mat-icon-button="" type="button" class="mat-mdc-tooltip-trigger mat-mdc-paginator-navigation-next mdc-icon-button mat-mdc-icon-button mat-unthemed mat-mdc-button-base cdk-focused cdk-mouse-focused" mat-ripple-loader-class-name="mat-mdc-button-ripple" mat-ripple-loader-centered="" aria-label="Página siguiente">...</button> is not clickable at point (1252, 358). Other element would receive the click: <div _ngcontent-ng-c2675381852="" class="loading-overlay ng-tns-c2675381852-0 ng-star-inserted">...</div>
  (Session info: chrome=126.0.6478.63)
Stacktrace:
	GetHandleVerifier [0x007E0ED3+27667]
	(No symbol) [0x00776BD4]
	(No symbol) [0x00671AFF]
	(No symbol) [0x006B8BB2]
	(No symbol) [0x006B6F3D]
	(No symbol) [0x006B4DCB]
	(No symbol) [0x006B4377]
	(No symbol) [0x006A911F]
	(No symbol) [0x006D392C]
	(No symbol) [0x006A8BA5]
	(No symbol) [0x006D3BC4]
	(No symbol) [0x006ECAC7]
	(No symbol) [0x006D36C6]
	(No 

# PRUEBAS

In [45]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

# Conexión con la página web a "scrapear"
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=opts)
driver.get('https://procesosjudiciales.funcionjudicial.gob.ec/busqueda-filtros')


In [58]:
# Se obtiene el número limite para realizar los cambios de página
page_label = driver.find_element(By.XPATH, "//div[@class = 'mat-mdc-paginator-range-label']")
text = page_label.text

# Se utiliza la funcion split para obtener un array por palabra del texto obtenido y luego se obtiene el ultimo registro
parts = text.split()
limit = int(parts[len(parts) - 1])

### Función para obtener los detalles del proceso

In [59]:
# Obtención del detalle
# Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))

#print(listado_procesos[0].text)
def obtencion_detalle(proceso):
    
        # Se espera unos segundos antes de dar click al boton de detalles
        sleep(random.uniform(8.0,10.0))
        listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
        
        numero_proceso = proceso.find_element(By.XPATH,'.//div[@class="numero-proceso"]').text
        boton_proceso = proceso.find_element(By.XPATH,'.//a[@href="/movimientos"]')
        boton_proceso.click()
        
        #listado_detalles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[contains(@class = 'cabecera-tabla')]")))
        listado_detalles = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, \"lista-movimiento-individual\")]")))

        # Se envian los detalles a la base de datos
        enviar_detalles(numero_proceso,listado_detalles)